#### 1. Необходимо собрать информацию о вакансиях на должность программиста или разработчика с сайта job.ru или hh.ru. (Можно с обоих сразу) Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе:  
• Наименование вакансии,  
• Предлагаемую зарплату,   
• Ссылку на саму вакансию  

In [1]:
import pandas as pd
import requests
import json
import re
from bs4 import BeautifulSoup as bs

In [2]:
def get_html(link, headers):
    with requests.session() as s:
        r = s.get(link, headers=headers)
    if r.status_code == 200:
        return r.text
    else:
        return False

def get_salary_info(link, headers):
    html = get_html(link, headers)
    _min_salary, _max_salary = 0, 0
    if html:
        soup = bs(html, 'html.parser')
        tags = soup.find_all('p', class_='vacancy-salary')
        _salary = re.findall(r'\d+', re.sub(r'\s', '', bs(str(tags[0])).get_text()))
        if len(_salary) == 0:
            pass
        elif len(_salary) == 1:
            _min_salary = _max_salary = _salary[0]
        elif len(_salary) == 2:
            _min_salary,  _max_salary = _salary
        else:
            pass
        return _min_salary, _max_salary
    else:
        return _min_salary, _max_salary

def get_vacancy_info(search_queries=[], page_count=1, headers={'User-agent': 'Mozilla/5.0'}):
    if len(search_queries) > 0:
        _df_list = []
        for search_query in search_queries:
            text = '+'.join(search_query.split(' ')).capitalize()
            for i in range(page_count):
                link = f'https://hh.ru/search/vacancy?area=1&text={text}&page={i}'
                html = get_html(link, headers)
                if html:
                    soup = bs(html, 'html.parser')
                    tags = soup.find_all('a', class_='bloko-link HH-LinkModifier')
                    _vacancies = {
                        'id': [],
                        'name': [],
                        'min_salary': [],
                        'max_salary': [],
                        'link': []
                    }
                    for tag in tags:
                        _id = re.findall(r'(\d+)\??', tag['href'])[0]
                        _name = bs(str(tag)).get_text()
                        _min_salary, _max_salary = get_salary_info(tag['href'], headers)
                        _link = tag['href']
                        _vacancies['id'].append(_id)
                        _vacancies['name'].append(_name)
                        _vacancies['min_salary'].append(_min_salary)
                        _vacancies['max_salary'].append(_max_salary)
                        _vacancies['link'].append(_link)
                    _df_temp = pd.DataFrame(_vacancies)
                else:
                    return
                _df_list.append(_df_temp)
        return pd.concat(_df_list).reset_index(drop=True)
    else:
        return

In [3]:
df = get_vacancy_info(search_queries=['программист', 'разработчик'], page_count=3)

In [4]:
df.head(10)

,id,name,min_salary,max_salary,link
0,32859767,Full-Stack Разработчик JavaScript (Node.js + A...,120000,120000,https://hh.ru/vacancy/32859767?query=%D0%9F%D1...
1,33036570,Программист (C#\ ASP.NET\MVC),100000,100000,https://hh.ru/vacancy/33036570?query=%D0%9F%D1...
2,32728996,Web-программист/PHP-программист,0,0,https://hh.ru/vacancy/32728996?query=%D0%9F%D1...
3,32666256,Программист C/С++,80000,120000,https://hh.ru/vacancy/32666256?query=%D0%9F%D1...
4,32117045,Программист JS (WebVR/WebAR),80000,150000,https://hh.ru/vacancy/32117045?query=%D0%9F%D1...
5,32379471,Программист Android,70000,120000,https://hh.ru/vacancy/32379471?query=%D0%9F%D1...
6,33078382,Backend разработчик (PHP программист),0,0,https://hh.ru/vacancy/33078382?query=%D0%9F%D1...
7,31795936,Backend разработчик (Java),0,0,https://hh.ru/vacancy/31795936?query=%D0%9F%D1...
8,32168891,"Middle веб-программист (PHP, ExtJs)",0,0,https://hh.ru/vacancy/32168891?query=%D0%9F%D1...
9,31794578,Frontend-разработчик (React),0,0,https://hh.ru/vacancy/31794578?query=%D0%9F%D1...


#### 2. Доработать приложение таким образом, чтобы можно было искать разработчиков на разные языки программирования (Например Python, Java, C++)

In [5]:
df = get_vacancy_info(search_queries=['Разработчик Python', 'Разработчик Java', 'Разработчик C++'], page_count=3)

In [6]:
df.head(10)

,id,name,min_salary,max_salary,link
0,33077842,Разработчик Tableau,0,0,https://hh.ru/vacancy/33077842?query=%D0%A0%D0...
1,33054998,"Senior python developer в Трипстер (Python, Dj...",200000,200000,https://hh.ru/vacancy/33054998?query=%D0%A0%D0...
2,31938413,Стажер-разработчик бекэнда (С++/Python),0,0,https://hh.ru/vacancy/31938413?query=%D0%A0%D0...
3,32537820,Стажер-разработчик (машинное обучение),0,0,https://hh.ru/vacancy/32537820?query=%D0%A0%D0...
4,32144172,"Senior Front-end Developer (JavaScript, React)",200000,200000,https://hh.ru/vacancy/32144172?query=%D0%A0%D0...
5,32933934,Ведущий python-разработчик,120000,120000,https://hh.ru/vacancy/32933934?query=%D0%A0%D0...
6,32702965,Senior Backend разработчик Python/C++ в Сан-Фр...,300000,300000,https://hh.ru/vacancy/32702965?query=%D0%A0%D0...
7,32855887,Python разработчик,0,0,https://hh.ru/vacancy/32855887?query=%D0%A0%D0...
8,33084177,Программист Python (Django),130000,180000,https://hh.ru/vacancy/33084177?query=%D0%A0%D0...
9,32922843,"Разработчик (Python,Django)",0,0,https://hh.ru/vacancy/32922843?query=%D0%A0%D0...
